In [1]:
import os, time, shutil
from pathlib import Path
from glob import glob
import numpy as np
import spikeinterface.full as si

In [2]:
def clean_sorting(rec, save_root, stream_name):
    
    # Creates output folder, sorts and clears large temporary files
    save_path = os.path.join(save_root,stream_name)
    output_folder = Path(os.path.join(save_path, 'sorted'))
    
    if not os.path.exists(os.path.join(output_folder,'amplitudes.npy')):
        output_folder.mkdir(parents=True, exist_ok=True)
        raw_file = os.path.join(output_folder,'recording.dat')
        wh_file = os.path.join(output_folder, 'temp_wh.dat')
        
        print(f"DURATION: {rec.get_num_frames() / rec.get_sampling_frequency()} s -- "
                f"NUM. CHANNELS: {rec.get_num_channels()}")
        
        try:
            t_start_sort = time.time()
            sorting = si.run_sorter(sorter, rec, output_folder=output_folder, verbose=True,
                                    **sorter_params)
            print(f"\n\nSpike sorting elapsed time {time.time() - t_start_sort} s")
            
            #Making sure we clean up the temporary .npy files
            if os.path.exists(wh_file):
                os.remove(wh_file)
            if os.path.exists(raw_file):
                os.remove(raw_file)
        except Exception:
            if os.path.exists(wh_file):
                os.remove(wh_file)
            if os.path.exists(raw_file):
                os.remove(raw_file)

In [3]:
sorter = 'kilosort2_5'
si.Kilosort2_5Sorter.set_kilosort2_5_path('/home/phornauer/Git/Kilosort_2020b')
sorter_params = si.get_default_sorter_params(si.Kilosort2_5Sorter)
sorter_params['n_jobs'] = -1
sorter_params['detect_threshold'] = 5.5
sorter_params['minFR'] = 0.01
sorter_params['minfr_goodchannels'] = 0.01
sorter_params['keep_good_only'] = False
sorter_params['do_correction'] = False

Setting KILOSORT2_5_PATH environment variable for subprocess calls to: /home/phornauer/Git/Kilosort_2020b


In [21]:
N_recordings = 3 # Number of recordings per chip/well that need to exist
parent_path = '/net/bs-filesvr02/export/group/hierlemann/recordings/Mea1k/phornauer/'
recording_date = '2301*'
project_name = 'Quinpirole'
chip_id = '*'

assay_name = 'Network'
assay_id = '00*'
file_name = 'data.raw.h5'

path_parts = [parent_path, recording_date, project_name, chip_id]
path_pattern = os.path.join(*path_parts)
path_list  = glob(path_pattern)

In [23]:
path_list

['/net/bs-filesvr02/export/group/hierlemann/recordings/Mea1k/phornauer/230119/Quinpirole/M05579',
 '/net/bs-filesvr02/export/group/hierlemann/recordings/Mea1k/phornauer/230119/Quinpirole/M05854',
 '/net/bs-filesvr02/export/group/hierlemann/recordings/Mea1k/phornauer/230119/Quinpirole/M05706',
 '/net/bs-filesvr02/export/group/hierlemann/recordings/Mea1k/phornauer/230119/Quinpirole/M04258',
 '/net/bs-filesvr02/export/group/hierlemann/recordings/Mea1k/phornauer/230119/Quinpirole/M04163',
 '/net/bs-filesvr02/export/group/hierlemann/recordings/Mea1k/phornauer/230118/Quinpirole/M05856',
 '/net/bs-filesvr02/export/group/hierlemann/recordings/Mea1k/phornauer/230118/Quinpirole/M05867',
 '/net/bs-filesvr02/export/group/hierlemann/recordings/Mea1k/phornauer/230118/Quinpirole/M04258',
 '/net/bs-filesvr02/export/group/hierlemann/recordings/Mea1k/phornauer/230118/Quinpirole/M04284',
 '/net/bs-filesvr02/export/group/hierlemann/recordings/Mea1k/phornauer/230118/Quinpirole/M05865',
 '/net/bs-filesvr02/

In [24]:
for chip_path in path_list: # Iterate over all chip IDs
    
    chip_pattern = os.path.join(chip_path, assay_name, assay_id, file_name)
    recording_path_list = glob(chip_pattern)
    recording_path_list.sort() # Make sure assay IDs are in the correct order
    
    for well_id in range(6): # Iterate over all wells
        stream_name = 'well00' + str(well_id)
        
        path_parts = chip_path.split('/')
        path_parts[0] = '/'
        path_parts[6] = 'intermediate_data'
        save_root = os.path.join(*path_parts[0:12])
        
        well_recording_list = []
        
        for rec_path in recording_path_list: #Iterate over recordings to be concatenated 
            
            try: # If not all wells were recorded, should be the only cause for an error
                rec = si.MaxwellRecordingExtractor(rec_path,stream_name=stream_name)
                well_recording_list.append(rec)
            except Exception:
                continue
        
        if len(well_recording_list) >= N_recordings:
            multirecording = si.concatenate_recordings(well_recording_list[:3])
            clean_sorting(multirecording, save_root, stream_name)
            

In [9]:
chip_path = path_list[0]
chip_pattern = os.path.join(chip_path, assay_name, assay_id, file_name)
recording_path_list = glob(chip_pattern)
recording_path_list.sort() # Make sure assay IDs are in the correct order

In [13]:
well_id = 0
stream_name = 'well00' + str(well_id)
        
path_parts = chip_path.split('/')
path_parts[0] = '/'
path_parts[6] = 'intermediate_data'
save_root = os.path.join(*path_parts[0:12])

well_recording_list = []
rec_path = recording_path_list[0]

In [14]:
save_root

'/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Mea1k/phornauer/230203/How_quinpirole/M05569'